<a href="https://colab.research.google.com/github/snwnkang/CS171_Final_vis_Marathon/blob/main/Boston_marathon_scraper2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from IPython.display import HTML
import html

In [2]:
year = 2017

In [3]:
df = pd.read_csv(f"https://raw.githubusercontent.com/snwnkang/CS171_Final_vis_Marathon/main/data/boston_data/results{year}.csv", error_bad_lines=False)

<ipython-input-3-ed40e4eddf86>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(f"https://raw.githubusercontent.com/snwnkang/CS171_Final_vis_Marathon/main/data/boston_data/results{year}.csv", error_bad_lines=False)


In [4]:
df.head(5)

,place_overall,bib,name,age,gender,city,state,country_residence,contry_citizenship,name_suffix,...,pace,projected_time,official_time,overall,gender_result,division_result,seconds,first_name,last_name,display_name
0,0.0,11,"Kirui, Geoffrey",24.0,M,Keringet,NaN,KEN,NaN,NaN,...,0:04:57,-,2:09:37,1.0,1.0,1.0,7777,Geoffrey,Kirui,Geoffrey Kirui
1,1.0,17,"Rupp, Galen",30.0,M,Portland,OR,USA,NaN,NaN,...,0:04:58,-,2:09:58,2.0,2.0,2.0,7798,Galen,Rupp,Galen Rupp
2,2.0,23,"Osako, Suguru",25.0,M,Machida-City,NaN,JPN,NaN,NaN,...,0:04:59,-,2:10:28,3.0,3.0,3.0,7828,Suguru,Osako,Suguru Osako
3,3.0,21,"Biwott, Shadrack",32.0,M,Mammoth Lakes,CA,USA,NaN,NaN,...,0:05:03,-,2:12:08,4.0,4.0,4.0,7928,Shadrack,Biwott,Shadrack Biwott
4,4.0,9,"Chebet, Wilson",31.0,M,Marakwet,NaN,KEN,NaN,NaN,...,0:05:04,-,2:12:35,5.0,5.0,5.0,7955,Wilson,Chebet,Wilson Chebet


In [5]:
columns_to_keep = ['place_overall', 'bib', 'name', 'age', 'gender', 'country_residence', 'official_time', 'seconds']

# Drop all columns except the ones specified
df = df[columns_to_keep]

# Now boston2019 contains only the columns you want to keep
print(df.head())

   place_overall bib              name   age gender country_residence  \
0            0.0  11   Kirui, Geoffrey  24.0      M               KEN   
1            1.0  17       Rupp, Galen  30.0      M               USA   
2            2.0  23     Osako, Suguru  25.0      M               JPN   
3            3.0  21  Biwott, Shadrack  32.0      M               USA   
4            4.0   9    Chebet, Wilson  31.0      M               KEN   

  official_time  seconds  
0       2:09:37     7777  
1       2:09:58     7798  
2       2:10:28     7828  
3       2:12:08     7928  
4       2:12:35     7955  


In [6]:
df.head(5)

,place_overall,bib,name,age,gender,country_residence,official_time,seconds
0,0.0,11,"Kirui, Geoffrey",24.0,M,KEN,2:09:37,7777
1,1.0,17,"Rupp, Galen",30.0,M,USA,2:09:58,7798
2,2.0,23,"Osako, Suguru",25.0,M,JPN,2:10:28,7828
3,3.0,21,"Biwott, Shadrack",32.0,M,USA,2:12:08,7928
4,4.0,9,"Chebet, Wilson",31.0,M,KEN,2:12:35,7955


In [7]:
def categorize_age(age):
    if 18 <= age <= 19:
        return '18-19'
    elif 20 <= age <= 24:
        return '20-24'
    elif 25 <= age <= 29:
        return '25-29'
    elif 30 <= age <= 34:
        return '30-34'
    elif 35 <= age <= 39:
        return '35-39'
    elif 40 <= age <= 44:
        return '40-44'
    elif 45 <= age <= 49:
        return '45-49'
    elif 50 <= age <= 54:
        return '50-54'
    elif 55 <= age <= 59:
        return '55-59'
    elif 60 <= age <= 64:
        return '60-64'
    elif 65 <= age <= 69:
        return '65-69'
    else:
        return '70+'  # For all ages above 69

In [14]:
unique_ages = df['age'].unique()
unique_ages

array([24., 30., 25., 32., 31., 40., 33., 28., 27., 41., 34., 29., 37.,
       42., 36., 39., 45., 26., 35., 19., 43., 23., 48., 21., 22., 20.,
       44., 46., 38., 50., 49., 47., 53., 56., 52., 55., 57., 59., 51.,
       58., 54., 60., 61., 69., 63., 18., 65., 62., 64., 67., 66., 70.,
       68., 77., 71., 72., 74., 79., 76., 73., 78., 80., 75., 81., 83.,
       84.])

In [9]:
df = df[df['age'] != 'M']

In [13]:
df = df.dropna(subset=['age'])

In [15]:
df['age'] = df['age'].astype(int)

In [16]:
df['Division'] = df['age'].apply(categorize_age)
df

,place_overall,bib,name,age,gender,country_residence,official_time,seconds,Division
0,0.0,11,"Kirui, Geoffrey",24,M,KEN,2:09:37,7777,20-24
1,1.0,17,"Rupp, Galen",30,M,USA,2:09:58,7798,30-34
2,2.0,23,"Osako, Suguru",25,M,JPN,2:10:28,7828,25-29
3,3.0,21,"Biwott, Shadrack",32,M,USA,2:12:08,7928,30-34
4,4.0,9,"Chebet, Wilson",31,M,KEN,2:12:35,7955,30-34
...,...,...,...,...,...,...,...,...,...
26405,26405.0,25166,"Steinbach, Paula Eyvonne",61,F,USA,7:09:39,25779,60-64
26406,26406.0,25178,"Avelino, Andrew R.",25,M,USA,7:16:59,26219,25-29
26407,26407.0,27086,"Hantel, Johanna",57,F,USA,7:19:37,26377,55-59
26408,26408.0,25268,"Reilly, Bill",64,M,USA,7:20:44,26444,60-64


In [17]:
def split_names(name):
    # Split the name at the comma
    parts = name.split(',')
    # The first part is the last name, the second part is the first name
    last_name = parts[0].strip()
    first_name = parts[1].strip() if len(parts) > 1 else ''
    return last_name, first_name

# Apply the function to the 'name' column and create new columns for the split names
df[['Last Name', 'First Name']] = df['name'].apply(
    lambda x: pd.Series(split_names(x), index=['Last Name', 'First Name'])
)

# Display the DataFrame to verify the changes
df[['name', 'Last Name', 'First Name']].head()

,name,Last Name,First Name
0,"Kirui, Geoffrey",Kirui,Geoffrey
1,"Rupp, Galen",Rupp,Galen
2,"Osako, Suguru",Osako,Suguru
3,"Biwott, Shadrack",Biwott,Shadrack
4,"Chebet, Wilson",Chebet,Wilson


In [18]:
df

,place_overall,bib,name,age,gender,country_residence,official_time,seconds,Division,Last Name,First Name
0,0.0,11,"Kirui, Geoffrey",24,M,KEN,2:09:37,7777,20-24,Kirui,Geoffrey
1,1.0,17,"Rupp, Galen",30,M,USA,2:09:58,7798,30-34,Rupp,Galen
2,2.0,23,"Osako, Suguru",25,M,JPN,2:10:28,7828,25-29,Osako,Suguru
3,3.0,21,"Biwott, Shadrack",32,M,USA,2:12:08,7928,30-34,Biwott,Shadrack
4,4.0,9,"Chebet, Wilson",31,M,KEN,2:12:35,7955,30-34,Chebet,Wilson
...,...,...,...,...,...,...,...,...,...,...,...
26405,26405.0,25166,"Steinbach, Paula Eyvonne",61,F,USA,7:09:39,25779,60-64,Steinbach,Paula Eyvonne
26406,26406.0,25178,"Avelino, Andrew R.",25,M,USA,7:16:59,26219,25-29,Avelino,Andrew R.
26407,26407.0,27086,"Hantel, Johanna",57,F,USA,7:19:37,26377,55-59,Hantel,Johanna
26408,26408.0,25268,"Reilly, Bill",64,M,USA,7:20:44,26444,60-64,Reilly,Bill


In [19]:
df['Year'] = year

In [20]:
rename_columns = {
    'place_overall': 'Overall Place',
    'bib': 'BIB',
    'country_residence': 'Country',
    'gender': 'Gender',
    'official_time': 'Time',
    'seconds': 'Seconds'
}

df.rename(columns=rename_columns, inplace=True)

In [21]:
new_column_order = [
    'Last Name', 'First Name', 'Division', 'Country', 'BIB', 'Overall Place',
    'Time', 'Seconds', 'Year', 'Gender'
]

df = df[new_column_order]

In [22]:
unique_country = df['Country'].unique()
unique_country

array(['KEN', 'USA', 'JPN', 'ETH', 'ZIM', 'MEX', 'BRN', 'BRA', 'CAN',
       'ITA', 'NED', 'IRL', 'BDI', 'GBR', 'GER', 'ESP', 'SUI', 'COL',
       'NOR', 'POL', 'FRA', 'RUS', 'POR', 'CRC', 'BEL', 'NZL', 'PER',
       'DOM', 'PAN', 'AUS', 'DEN', 'MAS', 'PHI', 'CHI', 'AUT', 'ARG',
       'SIN', 'CHN', 'TWN', 'GUA', 'RSA', 'HKG', 'SLO', 'EST', 'TUR',
       'EGY', 'IND', 'UAE', 'SWE', 'BER', 'KOR', 'CAY', 'FIN', 'LTU',
       'GRE', 'CRO', 'ISL', 'LUX', 'FLK', 'VEN', 'JAM', 'THA', 'HON',
       'SRB', 'CZE', 'ISR', 'MLT', 'HUN', 'ECU', 'KSA', 'ESA', 'MAR',
       'ROU', 'LAT', 'UKR', 'SMR', 'BUL', 'ALG', 'NCA', 'SVK', 'AND',
       'BAR', 'INA', 'GRN', 'URU', 'PAR', 'TCA', 'KUW', 'TRI', 'MGL',
       'NGR'], dtype=object)

In [64]:
country_codes = {
    'Kenya': 'KEN', 'Ethiopi': 'ETH', 'United ': 'USA', 'Japan': 'JPN',
    'Morocco': 'MAR', 'Mexico': 'MEX', 'Canada': 'CAN', 'Austral': 'AUS',
    'Germany': 'DEU', 'China': 'CHN', 'Dominic': 'DMA', 'Israel': 'ISR',
    'Argenti': 'ARG', 'Ireland': 'IRL', 'Brazil': 'BRA', 'Turkey': 'TUR',
    'Austria': 'AUT', 'Peru': 'PER', 'France': 'FRA', 'Chile': 'CHL',
    'Switzer': 'CHE', 'Denmark': 'DNK', 'Norway': 'NOR', 'Finland': 'FIN',
    'Portuga': 'PRT', 'Sweden': 'SWE', 'Latvia': 'LVA', 'Colombi': 'COL',
    'Bermuda': 'BMU', 'Belgium': 'BEL', 'Hong Ko': 'HKG', 'Ecuador': 'ECU',
    'Italy': 'ITA', 'Costa R': 'CRI', 'Taiwan': 'TWN', 'Estonia': 'EST',
    'Spain': 'ESP', 'Palesti': 'PSE', 'Venezue': 'VEN', 'Czech R': 'CZE',
    'Netherl': 'NLD', 'Russian': 'RUS', 'India': 'IND', 'South A': 'ZAF',
    'Singapo': 'SGP', 'Poland': 'POL', 'Luxembo': 'LUX', 'Hungary': 'HUN',
    'Guatema': 'GTM', 'Lithuan': 'LTU', 'Ukraine': 'UKR', 'Belarus': 'BLR',
    'Sloveni': 'SVN', 'Korea, ': 'KOR', 'New Zea': 'NZL', 'Puerto ': 'PRI',
    'Macao': 'MAC', 'Armenia': 'ARM', 'El Salv': 'SLV', 'Thailan': 'THA',
    'Nepal': 'NPL', 'Philipp': 'PHL', 'Slovaki': 'SVK', 'Vietnam': 'VNM',
    'Faroe I': 'FRO', 'Panama': 'PAN', 'Andorra': 'AND', 'Hondura': 'HND',
    'Uruguay': 'URY', 'Malaysi': 'MYS', 'Iceland': 'ISL', 'Monaco': 'MCO',
    'Indones': 'IDN', 'Trinida': 'TTO', 'Algeria': 'DZA', 'Cyprus': 'CYP',
    'Greece': 'GRC', 'Cayman ': 'CYM', 'Moldova': 'MDA', 'Romania': 'ROU',
    'Serbia ': 'SRB', 'Croatia': 'HRV', 'Belize': 'BLZ', 'Saudi A': 'SAU',
    'Malta': 'MLT', 'Kuwait': 'KWT', 'Bolivia': 'BOL', 'Barbado': 'BRB',
    'Martini': 'MTQ', 'Aruba': 'ABW', 'Macedon': 'MKD', 'Bahrain': 'BHR', 'Oman': 'OMN',
    'Jamaica': 'JAM', 'Paragua': 'PRY', 'Brunei ': 'BRN', 'Jordan': 'JOR', 'GRC': 'GRC',
    'Macao S': 'MAC', 'Liechte': 'LIE', 'Bosnia ': 'BIH', 'Bahamas': 'BHS', 'Uganda': 'UGA',
    'Sri Lan': 'LKA'
}

In [65]:
def replace_country_names(df, country_dict):
    df['Country'] = df['Country'].map(country_dict).fillna(df['Country'])
    return df

In [66]:
df = replace_country_names(df, country_codes)
df.head()

<ipython-input-65-d5faa6fe5beb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].map(country_dict).fillna(df['Country'])


,Last Name,First Name,Division,Country,BIB,Overall Place,Time,Seconds,Year,Gender
0,Kawauchi,Yuki,30-34,JPN,NaN,1,2:15:58,8158,2018,M
1,Kirui,Geoffrey,25-29,KEN,NaN,2,2:18:23,8303,2018,M
2,Biwott,Shadrack,30-34,USA,NaN,3,2:18:35,8315,2018,M
3,Pennel,Tyler,30-34,USA,NaN,4,2:18:57,8337,2018,M
4,Bumbalough,Andrew,30-34,USA,NaN,5,2:19:52,8392,2018,M


In [67]:
unique_country = df['Country'].unique()
unique_country

array(['JPN', 'KEN', 'USA', 'NLD', 'CAN', 'CHE', 'CHN', 'TUR', 'NOR',
       'BEL', 'IRL', 'GTM', 'MEX', 'COL', 'LTU', 'AUS', 'DEU', 'PER',
       'FRA', 'ECU', 'ESP', 'CRI', 'CZE', 'BRA', 'RUS', 'SWE', 'ITA',
       'FIN', 'DNK', 'POL', 'ARG', 'HND', 'MCO', 'CHL', 'HUN', 'HKG',
       'ZAF', 'TWN', 'SVK', 'PRT', 'MTQ', 'SVN', 'AUT', 'NZL', 'IND',
       'DMA', 'MKD', 'SGP', 'BMU', 'CYP', 'SLV', 'TTO', 'EST', 'KOR',
       'BHR', 'ISL', 'PAN', 'OMN', 'BLR', 'JAM', 'MYS', 'LVA', 'MAR',
       'ISR', 'PRY', 'THA', 'BRN', 'JOR', 'VEN', 'GRC', 'MLT', 'SRB',
       'PRI', 'ROU', 'URY', 'NPL', 'VNM', 'MAC', 'HRV', 'CYM', 'LIE',
       'LUX', 'BIH', 'IDN', 'BHS', 'BRB', 'PHL', 'UGA', 'LKA'],
      dtype=object)

In [69]:
df
#up to here is the full data
df.to_csv(f"full{year}boston.csv")

In [70]:
first_100_M = df[df['Gender'] == 'M'].head(100)

first_100_F = df[df['Gender'] == 'F'].head(100)

In [71]:
first_100_M.to_csv(f"Boston{year}_M.csv")

In [72]:
first_100_F.to_csv(f"Boston{year}_F.csv")